In [1]:
!pip install tqdm

In [ ]:
# Importar las bibliotecas necesarias
import requests
from bs4 import BeautifulSoup

from google.colab import drive
import time
import random
from tqdm.notebook import tqdm

In [ ]:
# Montar Google Drive
drive.mount('/content/drive')

# Función para extraer información de una tarjeta
def extract_card_info(card):
    tag = card.select_one('.card__topics a').text.strip()
    title_element = card.select_one('.card__title a')
    title = title_element.text.strip()
    url = "https://www.infoq.com" + title_element['href'].split('?')[0]  # Extraer la URL base sin parámetros
    detail = card.select_one('.card__excerpt').text.strip()
    date = card.select_one('.card__date span').text.strip()
    author = card.select_one('.card__authors a').text.strip()
    return {
        'Tag': tag,
        'Title': title,
        'URL': url,
        'Detail': detail,
        'Date': date,
        'Author': author
    }

def scrape_infoq():
    base_url = "https://www.infoq.com/architecture-design/news/"
    csv_path = '/content/drive/MyDrive/infoq_architecture_design_news.csv'

    # Verificar si el CSV existe y obtener el último offset procesado
    try:
        df_existing = pd.read_csv(csv_path)
        last_processed_offset = len(df_existing) - (len(df_existing) % 15)
    except FileNotFoundError:
        last_processed_offset = 0

    # Calcular el número total de páginas
    total_pages = (8790 // 15) + 1

    # Crear la barra de progreso
    with tqdm(total=total_pages, initial=last_processed_offset//15, desc="Scraping Progress") as pbar:
        for offset in range(last_processed_offset, 8791, 15):
            url = f"{base_url}{offset}/" if offset > 0 else base_url

            try:
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')

                cards = soup.select('div.card__content')

                page_data = []
                for card in cards:
                    card_info = extract_card_info(card)
                    page_data.append(card_info)

                # Convertir los datos de la página a DataFrame
                df_page = pd.DataFrame(page_data)

                # Guardar o anexar al CSV
                if offset == 0 and last_processed_offset == 0:
                    df_page.to_csv(csv_path, index=False, mode='w')
                else:
                    df_page.to_csv(csv_path, index=False, mode='a', header=False)

                # Actualizar la barra de progreso con información adicional
                pbar.set_postfix_str(f"Page: {offset//15 + 1}, URL: {url}")
                pbar.update(1)

            except Exception as e:
                print(f"Error processing page {url}: {e}")

            # Pausa aleatoria para evitar sobrecarga del servidor
            time.sleep(random.uniform(1, 3))

    print(f"Scraping completo. CSV guardado en: {csv_path}")

# Cargar el CSV final y mostrar información
df = pd.read_csv('/content/drive/MyDrive/infoq_architecture_design_news.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Mostrar información sobre el DataFrame
print(df.info())

In [ ]:
# Montar Google Drive
from google.colab import drive
import pandas as pd

# Montar Google Drive
drive.mount('/content/drive')

# Ruta del archivo CSV original en Google Drive
csv_path = '/content/drive/MyDrive/infoq_architecture_design_news.csv'

# Cargar el archivo CSV
df = pd.read_csv(csv_path)

# Eliminar filas donde Tag, Author y Date sean NaN
df_cleaned = df.dropna(subset=['Tag', 'Author', 'Date'], how='all')

# Eliminar filas completamente vacías (que tienen solo valores NaN o están en blanco)
df_cleaned = df_cleaned.dropna(how='all')

# Guardar el DataFrame limpio en un nuevo archivo CSV
cleaned_csv_path = '/content/drive/MyDrive/infoq_architecture_design_news_cleaned.csv'
df_cleaned.to_csv(cleaned_csv_path, index=False)

# Mostrar las primeras filas del DataFrame limpio
print(df_cleaned.head(100))

print(f"CSV limpio guardado en: {cleaned_csv_path}")

In [ ]:
# Montar Google Drive
drive.mount('/content/drive')

# Cargar el CSV final y mostrar información
df = pd.read_csv('/content/drive/MyDrive/infoq_architecture_design_news.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Mostrar información sobre el DataFrame
print(df.info())

Mounted at /content/drive


NameError: name 'pd' is not defined